Основная версия 5 августа 2020. Все пути захардкожены, в будущем нужно поменять.  
Версия 12 августа 2020. Написана функция download_data.

In [1]:
import os
import sys
import re
import json
import wget
import pyunpack
import fileinput
import glob
from itertools import chain
from ast import literal_eval
from tqdm import tqdm
import numpy as np
from numpy.random import choice
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers.tokenization_bert import BasicTokenizer
pd.set_option('display.max_colwidth', 500)

In [2]:
class SyntheticDatasetGenerator:
    def __init__(self, all_data, output_file, generate_type = ['tsya','nn', 'techenie', 'neni_b'], data_dir='data/'):
        self.data_dir = data_dir
        self.dataset_path = os.path.join(data_dir, all_data)
        #self.download_data(self.data_dir)
        
        self.input_num_sentences = None
        self.output_num_sentences = None
        self.words = []
        if 'tsya' in generate_type:
            self.tysya_existing_words = None
            self.tsya_existing_words = None
            self.words += ['tsya', 'tysya']
        if 'nn' in generate_type:
            self.n_nn_existing_words = None
            self.words += ['n', 'nn']
        if 'techenie' in generate_type:
            self.techenie_existing_words = None
            self.words += ['techenie', 'techenii']
        if 'neni_b' in generate_type:
            self.ne_ni_existing_words = None 
            self.words += ['ne', 'ni']
        self.generate_type = generate_type
        self.load_dics()
        self.taglist = ['REPLACE_tsya', 'REPLACE_tysya', 'REPLACE_nn, REPLACE_n', 'REPLACE_techenie', 'REPLACE_techenii'
                           'REPLACE_ne', 'REPLACE_ni']
        self.KEEP = 'O'
        self.to_file = output_file
        self.data = None
    
    def load_dics(self):
        print('Loading dictionaries...')
        
        if 'techenie' in self.generate_type:
            self.techenie_existing_words = set(['течение', 'течении'])
        if 'tsya' in self.generate_type:
            with open('data/tsya_vocab.txt', 'r') as f:
                pairs = f.read().splitlines()
            self.tysya_existing_words = set([pair.split('\t')[0] for pair in pairs])
            self.tsya_existing_words = set([pair.split('\t')[1] for pair in pairs])
        if 'nn' in self.generate_type:
            if not os.path.exists('data/all_n_nn_words_full_endings.txt'):
                with open('data/n_words.txt', 'r') as f:
                    n_words = f.read().splitlines()
                n_data = pd.DataFrame(n_words, columns=['text'])
                n_data['text'] = n_data['text'].str.replace('"Словоформа": "', '').str.strip('"').str.strip()
                # remove words with capital letters
                to_filter = n_data[n_data['text'].str.contains(r'[А-ЯЁ]+', regex=True)]
                #  keep those with capital letters which have a hyphen after them
                to_append = to_filter[to_filter['text'].str.contains(r'[А-ЯЁ]+-')]
                n_data = n_data[~n_data['text'].str.contains(r'[А-ЯЁ]+', regex=True)]
                n_data = n_data.append(to_append)
                nn_pattern = re.compile(r'\wнн([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
                n_pattern = re.compile(r'[аоэеиыуёюя]н([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
                nn = n_data[n_data['text'].str.contains(nn_pattern, regex=True)]
                n = n_data[n_data['text'].str.contains(n_pattern, regex=True)]
                self.n_nn_existing_words = set(nn['text']).union(set(n['text']))
                # add word writings with 'е' instead of 'ё'
                without_yo = set()
                for word in self.n_nn_existing_words:
                    if word.find('ё') != -1:
                        without_yo.add(word.replace('ё', 'е'))
                self.n_nn_existing_words.update(without_yo)
                # write n_nn_words to disk for further using out of the file
                with open('data/all_n_nn_words_full_endings.txt', 'w') as f:
                    for line in self.n_nn_existing_words:
                        f.write(line + '\n')
            else:
                with open('data/all_n_nn_words_full_endings.txt', 'r') as f:
                    self.n_nn_existing_words = set(f.read().splitlines())
        if 'neni_b' in self.generate_type:        
            if not os.path.exists('data/all_ne_ni_words.txt'):
                with open('data/ne_ni_words.txt', 'r') as f:
                    ne_ni_words = f.read().splitlines()
                ne_ni_data = pd.DataFrame(ne_ni_words, columns=['text'])
                ne_ni_data['text'] = ne_ni_data['text'].str.replace('"Словоформа": "', '').str.strip('"').str.strip()
                to_filter = ne_ni_data[ne_ni_data['text'].str.contains(r'[А-ЯЁ]+', regex=True)]
                to_append = to_filter[to_filter['text'].str.contains(r'[А-ЯЁ]+-')]
                ne_ni_data = ne_ni_data[~ne_ni_data['text'].str.contains(r'[А-ЯЁ]+', regex=True)]
                ne_ni_data = ne_ni_data.append(to_append)
                ne_pattern = re.compile(r'\b([А-ЯЁ]+-)?не\w+(\b|-)', re.IGNORECASE)
                ni_pattern = re.compile(r'\b([А-ЯЁ]+-)?ни\w+(\b|-)', re.IGNORECASE)
                ne = ne_ni_data[ne_ni_data['text'].str.contains(ne_pattern, regex=True)]
                ni = ne_ni_data[ne_ni_data['text'].str.contains(ni_pattern, regex=True)]
                bool_signs = ne['text'].str.replace('не', 'ни').isin(ni['text'])
                ne_ni_existing_words = set(ne[bool_signs]['text']).union(set(ne[bool_signs]['text'].str.replace('не', 'ни')))
                without_yo = set()
                for word in ne_ni_existing_words:
                    if word.find('ё') != -1:
                        without_yo.add(word.replace('ё', 'е'))
                ne_ni_existing_words.update(without_yo)
                with open('data/all_ne_ni_words.txt', 'w') as f:
                    for line in ne_ni_existing_words:
                        f.write(line + '\n')
            else:
                with open('data/all_ne_ni_words.txt', 'r') as f:
                    self.ne_ni_existing_words = set(f.read().splitlines())
            
            
    def download_data(self, data_dir):
        print('Downloading data...')
        if not os.path.exists(data_dir):
            os.mkdir(data_dir)
        dataset_name = 'Magazines'
        if os.path.exists(self.dataset_path):
            print('all_data.txt already exists')
        else:
            archive_path = os.path.join(data_dir, dataset_name + '.rar')
            if not os.path.exists(archive_path):
                archive_url = 'https://linghub.ru/static/Taiga/' + dataset_name +'.rar'
                filename = wget.download(archive_url, out=archive_path)
            unrared_path = os.path.join(data_dir, dataset_name)
            if not os.path.exists(unrared_path):            
                pyunpack.Archive(archive_path).extractall(data_dir)
            file_list = sorted(glob.glob(os.path.join(unrared_path, 'texts/*.txt')))
            with open(self.dataset_path, 'w') as file:
                input_lines = fileinput.input(file_list)
                file.writelines(input_lines)

        n_words_path = os.path.join(data_dir, 'n_words.txt')
        if not os.path.exists(n_words_path):
            all_orpho_words_path = os.path.join(data_dir, 'db_word_16.06.2020.json')
            if not os.path.exists(all_orpho_words_path):
                raise FileNotFoundError('Please put db_word_16.06.2020.json in ' + os.path.abspath(data_dir))
            with open(all_orpho_words_path, 'r') as f:
                with open(n_words_path, 'w') as writefile:
                    line = f.readline()
                    while line:
                        line = f.readline()
                        if re.search('н', line) and re.search('Словоформа', line):
                            writefile.write(line)
        
        tsya_vocab_path = os.path.join(data_dir, 'tsya_vocab.txt')
        if not os.path.exists(tsya_vocab_path):
            raise FileNotFoundError('Please put tsya_vocab.txt in ' + os.path.abspath(data_dir))
            
        ne_ni_words_path = os.path.join(data_dir, 'ne_ni_words.txt')
        if not os.path.exists(ne_ni_words_path):
            all_orpho_words_path = os.path.join(data_dir, 'db_word_16.06.2020.json')
            if not os.path.exists(all_orpho_words_path):
                raise FileNotFoundError('Please put db_word_16.06.2020.json in ' + os.path.abspath(data_dir))
            with open(all_orpho_words_path, 'r') as f:
                with open(ne_ni_words_path, 'w') as writefile:
                    line = f.readline()
                    while line:
                        line = f.readline()
                        if re.search(r'\b([А-ЯЁ]+-)?не', line) and re.search('Словоформа', line):
                            writefile.write(line)
                        elif re.search(r'\b(\b[А-ЯЁ]+-)?ни', line) and re.search('Словоформа', line):
                            writefile.write(line)
        # news_dataset      
#         archive_url = 'http://bit.ly/2pvhWZm'
#         filename = wget.download(archive_url, os.path.join('./data', 'news' + '.zip'))
#         pyunpack.Archive('./data/news.zip').extractall('./data')

#         file_list_full = []
#         for folder in ['Fontanka', 'Lenta', 'KP', 'Interfax']:
#             file_list_year = []
#             for year_folder in os.listdir('./news/' + folder + '/texts/'):
#                 file_list = sorted(glob.glob(os.path.join('./news/'+folder + '/texts/' + year_folder + '/', '*.txt')))
#                 file_list_year.append(file_list)
#             file_list_full.append(file_list_year)

#         with open('news_dataset.txt', 'w') as file:
#             for file_list_year in file_list_full:
#                 for file_list in file_list_year:
#                     input_lines = fileinput.input(file_list)
#                     file.writelines(input_lines)
        
    
    def select_data(self):
        print('Selecting lines larger than 10 symbols...')
        with open(self.dataset_path, 'r') as f:
            texts = f.read().splitlines() # 6697245 lines
            texts = [' '.join(line.split()) for line in texts if len(line) > 10] # 5014971 lines [5304067 lines]
        return texts
    
    def sentence_tokenize(self, texts):
        print('Splitting in sentences...')
        sentences = list(chain.from_iterable(list(map(sent_tokenize, texts))))
        self.input_num_sentences = len(sentences)
        print('Input sentences before splitting on ... :', self.input_num_sentences)
        new_sentences = []
        for s in sentences:
            longer = False
            sen = re.split(r'\.\.\.\s|…\s', s)
            if len(sen) > 1:
                longer = True
            for i in range(len(sen)):
                if longer and i < len(sen) - 1 and len(sen[i]) > 1:
                    new_sentences.append(sen[i] + '...')
                elif i == len(sen) - 1 and len(sen[i]) > 1:
                    new_sentences.append(sen[i])
        self.input_num_sentences = len(new_sentences)
        print('Input sentences:', self.input_num_sentences)
        return pd.DataFrame(new_sentences, columns=['text'])
    
    def filter_characters(self, data):
        print('Filtering characters...')
        # Heuristic for correct bert tokenization
        data['text'] = data['text'].str.replace(r'[“”«»‘‛’„»«└″′]', "'", regex=True)
        symbols_to_filter = set()
        for i, sent in tqdm(enumerate(data['text'])):
            for letter in sent:
                if ord(letter) > ord('─'):
                    symbols_to_filter.add(letter)
        symbols_to_filter.update({'\u200b','\u200e', '´', '`'})
        symbols_to_filter = ''.join(sym for sym in symbols_to_filter)
        filter_regex = re.compile(r'[' + re.escape(symbols_to_filter) + r']')
        data['text'] = data['text'].str.replace(filter_regex, '', regex=True)
        return data
    
    def add_schitannye(self, data):
        print("Add schitannye")
        schitan = re.compile(r'\bсчитан[ые|ыми|ое]', flags=re.IGNORECASE)
        schitann = re.compile(r'\bсчитанн[ые|ыми|ое]', flags=re.IGNORECASE)
        data['n_case'] = data['text'].str.contains(schitan, regex=True)
        data['schitanye'] = data[data['n_case'].fillna(False)]['text']
        data['nn_case']  = data['text'].str.contains(schitann, regex=True)
        data['schitannye'] = data[data['nn_case'].fillna(False)]['text']
        sizes = data.count().to_numpy()
        k = sizes[4]-sizes[2]
        print(k)
        schitan_data = data['schitanye'].dropna().to_numpy()
        base = k//sizes[2]*sizes[2]
        df = pd.concat([pd.DataFrame([schitan_data[i]], columns=['text']) for i in range(sizes[2])], ignore_index=True)
        df1 = pd.concat([data, df], ignore_index=True)
        for r in range(k//sizes[2]-1):
            df1 = pd.concat([df1, df], ignore_index=True)
        df2 = pd.concat([pd.DataFrame([schitan_data[i]], columns=['text']) for i in range(k-base)], ignore_index=True)
        data = pd.concat([df1, df2], ignore_index=True)     
        return data.drop(columns=['schitanye', 'schitannye', 'n_case', 'nn_case'])
    
    def insert_correct_schitanye(self, data):
        
        print("Insert correct sentences with считаные/считанные")
        schitan = re.compile(r'\bсчитан(?=[ые|ыми|ое])', flags=re.IGNORECASE)
        schitann = re.compile(r'\bсчитанн(?=[ые|ыми|ое])', flags=re.IGNORECASE)
        data['n_case'] = data['text'].str.contains(schitan, regex=True)
        data['nn_case'] = data['text'].str.contains(schitann, regex=True)
        data['schitanye'] = data[data['n_case'].fillna(False)]['text']
        data['schitannye'] = data[data['nn_case'].fillna(False)]['text']
        df = pd.concat([data['schitanye'].dropna(), data['schitannye'].dropna()]) 
        with open('schitanyj_1n.txt', 'r') as f:
            arr_n = f.readlines()
        arr_n = [line.strip() for line in arr_n]
        with open('schitannyj_2n.txt', 'r') as f:
            arr_nn = f.readlines()
        arr_nn = [line.strip() for line in arr_nn]
        for i,sentence in tqdm(df.iteritems()):
            if sentence in arr_n:
                #if schitann.search(data.loc[df.index[i], 'text']) is not None:
                if schitann.search(data.loc[i, 'text']) is not None:
                    word = schitann.search(data.loc[i, 'text']).group()
                    data.loc[i, 'text'] = data.loc[i, 'text'].replace(word, 'считан')
            if sentence in arr_nn:
                if schitan.search(data.loc[i, 'text']) is not None:
                    word = schitan.search(data.loc[i, 'text']).group()
                    data.loc[i, 'text'] = data.loc[i, 'text'].replace(word, 'считанн')
        data = data.drop(columns=['schitanye', 'schitannye', 'n_case', 'nn_case'])
        print("Data contains sentence, that is correct for both n and nn", data[data.text == "Они не могут быть считанными."]['text'])
        if "Они не могут быть считанными." in df.to_numpy():
            print("contains")
            data.loc[len(data), "text"] = "Они не могут быть считаными."
        for dset in [df, arr_n, arr_nn]:
            del dset
        return data
    
    def mark_containing_tsya_tysya(self, data):
        print('Marking tsya/tysya sentences...')
        data['tsya'] = data['text'].str.contains(r'\wтся', regex=True, flags=re.IGNORECASE)
        data['tysya'] = data['text'].str.contains(r'\wться', regex=True, flags=re.IGNORECASE)
        full_tsya_pattern = r'\b\w*тся'
        full_tysya_pattern = r'\b\w*ться'
        data['all_tsya_words'] = data[data['tsya'].fillna(False)]['text'].str.findall(full_tsya_pattern, flags=re.IGNORECASE)
        data['all_tysya_words'] = data[data['tysya'].fillna(False)]['text'].str.findall(full_tysya_pattern, flags=re.IGNORECASE)
        print(len(data[data['tsya'].fillna(False)]), len(data[data['tysya'].fillna(False)]))
        return data.drop(['tsya', 'tysya'], axis=1)

    def mark_containing_n_nn(self, data):
        print('Marking n/nn sentences...')
        nn_pattern = re.compile(r'\wнн([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
        n_pattern = re.compile(r'[аоэеиыуёюя]н([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
        data['nn'] = data['text'].str.contains(nn_pattern, regex=True)
        data['n'] = data['text'].str.contains(n_pattern, regex=True)
        full_n_pattern = r'\b\w*[аоэеиыуёюя]н(?:[аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b'
        data['all_n_words'] = data[data['n'].fillna(False)]['text'].str.findall(full_n_pattern, flags=re.IGNORECASE)
        full_nn_pattern = r'\b\w*нн(?:[аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b'
        data['all_nn_words'] = data[data['nn'].fillna(False)]['text'].str.findall(full_nn_pattern, flags=re.IGNORECASE)
        print(len(data[data['n'].fillna(False)]), len(data[data['nn'].fillna(False)]))
        return data.drop(['n', 'nn'], axis=1)
        
    
    def mark_containing_techenie_ii(self, data):
        print('Marking techenie/techenii sentences...')
        techenie_pattern = re.compile(r'\bтечение\b', re.IGNORECASE)
        techenii_pattern = re.compile(r'\bтечении\b', re.IGNORECASE)
        data['techenie'] = data['text'].str.contains(techenie_pattern, regex=True)
        data['techenii'] = data['text'].str.contains(techenii_pattern, regex=True)

        data['all_techenie_words'] = data[data['techenie'].fillna(False)]['text'].str.findall(r'\bтечение\b', flags=re.IGNORECASE)
        data['all_techenii_words'] = data[data['techenii'].fillna(False)]['text'].str.findall(r'\bтечении\b', flags=re.IGNORECASE)
        print(len(data[data['techenie'].fillna(False)]), len(data[data['techenii'].fillna(False)]))
        return data.drop(['techenie', 'techenii'], axis=1)
    
    def mark_containing_ne_ni(self, data):
        print('Marking ne/ni sentences...')
        nе_pattern = re.compile(r'\bне\w+\b', re.IGNORECASE)
        ni_pattern = re.compile(r'\bни\w+\b', re.IGNORECASE)
        data['ne'] = data['text'].str.contains(nе_pattern, regex=True)
        data['ni'] = data['text'].str.contains(ni_pattern, regex=True)

        data['all_ne_words'] = data[data['ne'].fillna(False)]['text'].str.findall(nе_pattern, flags=re.IGNORECASE)
        data['all_ni_words'] = data[data['ni'].fillna(False)]['text'].str.findall(ni_pattern, flags=re.IGNORECASE)
        print(len(data[data['ne'].fillna(False)]), len(data[data['ni'].fillna(False)]))
        return data.drop(['ne', 'ni'], axis=1)
    
    def generate_one_error_type(self, data, error_name):
        print('Generating error', error_name, '...')
        if error_name == 'tsya':
            pattern_cased = re.compile(r'ТСЯ')
            pattern = re.compile(r'тся', re.IGNORECASE)
            substitute_cased = 'ТЬСЯ'
            substitute = 'ться'
            existing_words = self.tysya_existing_words
        elif error_name == 'tysya':
            pattern_cased = re.compile(r'ТЬСЯ')
            pattern = re.compile(r'ться', re.IGNORECASE)
            substitute_cased = 'ТСЯ'
            substitute = 'тся'
            existing_words = self.tsya_existing_words
        elif error_name == 'n':
            pattern_cased = re.compile(r'(?<=[аоэеиыуёюя])(?-i:Н)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            pattern = re.compile(r'(?<=[аоэеиыуёюя])(?-i:н)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            substitute_cased = 'НН'
            substitute = 'нн'
            existing_words = self.n_nn_existing_words
        elif error_name == 'nn':
            pattern_cased = re.compile(r'(?-i:НН)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            pattern = re.compile(r'(?-i:нн)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            substitute_cased = 'Н'
            substitute = 'н'
            existing_words = self.n_nn_existing_words
        elif error_name == 'techenie':
            pattern_cased = re.compile(r'\bТЕЧЕНИЕ\b', re.IGNORECASE)
            pattern = re.compile(r'\bтечение\b', re.IGNORECASE)
            substitute_cased = 'ТЕЧЕНИИ'
            substitute = 'течении'
            existing_words = self.techenie_existing_words
        elif error_name == 'techenii':
            pattern_cased = re.compile(r'\bТЕЧЕНИИ\b', re.IGNORECASE)
            pattern = re.compile(r'\bтечении\b', re.IGNORECASE)
            substitute_cased = 'ТЕЧЕНИЕ'
            substitute = 'течение'
            existing_words = self.techenie_existing_words
        elif error_name == 'ne':
            pattern_cased = re.compile(r'\bНЕ\w+\b', re.IGNORECASE)
            pattern = re.compile(r'\bне\w+\b', re.IGNORECASE)
            substitute_cased = 'НИ'
            substitute = 'ни'
            existing_words = self.ni_existing_words
        elif error_name == 'ni':
            pattern_cased = re.compile(r'\bНИ\w+\b', re.IGNORECASE)
            pattern = re.compile(r'\bни\w+\b', re.IGNORECASE)
            substitute_cased = 'НЕ'
            substitute = 'не'
            existing_words = self.ne_existing_words
        original_array = []
        changed_array = []
        col_name = 'all_' + str(error_name) + '_words'
        for line in data[col_name]:
            if line is np.nan:
                changed_array.append(np.nan)
                original_array.append(np.nan)
            else:
                changed_line = []
                original_line = []
                for word in line:
                    # the order is important! first cased pattern, then pattern
                    changed_word = pattern_cased.sub(substitute_cased, word)
                    changed_word = pattern.sub(substitute, changed_word)
                    if changed_word.lower() in existing_words and changed_word != word:                        
                        changed_line.append(changed_word)
                        original_line.append(word)
                if len(changed_line) == 0:
                    changed_array.append(np.nan)
                    original_array.append(np.nan)
                else:
                    changed_array.append(changed_line)
                    original_array.append(original_line)
        data[str(error_name) + '_words'] = original_array
        data[str(error_name) + '_words_errorified'] = changed_array
        return data.drop('all_' + str(error_name) + '_words', axis=1)

    def filter_data_by_error_set(self, data):
        
        print('Filtering out the sentences that cannot be errorified...')
        potential_cols = []
        if 'tsya' in self.generate_type:
            potential_cols = potential_cols + ['tsya_words_errorified','tysya_words_errorified']
        if 'nn' in self.generate_type:
            potential_cols = potential_cols + ['n_words_errorified', 'nn_words_errorified']
        if 'techenie' in self.generate_type:
            potential_cols = potential_cols + ['techenie_words_errorified', 'techenii_words_errorified']
        if 'neni_b' in self.generate_type:
            potential_cols = potential_cols + ['ne_words_errorified', 'ni_words_errorified']

        #potential_cols = ['tsya_words_errorified', 'tysya_words_errorified', 'n_words_errorified', 'nn_words_errorified']
        data.dropna(subset=potential_cols, how='all', inplace=True)
        self.output_num_sentences = len(data)
        print('Output sentences:', self.output_num_sentences)   
        
        return data, potential_cols
    
    def errorify_sentences_new(self, data, potential_cols):
        
        print('Errorifying sentences...')
        np.random.seed(42)        
        cols = ['text'] + potential_cols + [word + '_words' for word in self.words]
        print(cols)
        potential_data = data[cols].copy()
        potential_data['error'] = np.nan
        potential_data['error_type'] = np.nan
        potential_data['error_word'] = np.nan
        potential_data.reset_index(drop=True, inplace=True)
        
        potential_data_correct = data[cols].copy()
        potential_data_correct['error'] = data['text'].copy()
        potential_data_correct['error_type'] = 'none'
        potential_data_correct['error_word'] = np.nan
        potential_data_correct.reset_index(drop=True, inplace=True)
        for i, row in tqdm(potential_data.iterrows()): # slow method, to be replaced
            cols_maybe_errors = [col for col in potential_cols if (row[col] and not np.all(pd.isnull(row[col])))]
            error_col = choice(cols_maybe_errors)
            # take randomly 1 possible incorrect word or none
            # change-in-future: p of 'no error' depends on the number of possible incorrect words
            potential_error_words = potential_data.loc[i, error_col]
            probabilities = np.empty(len(potential_error_words) + 1)
            probabilities[-1] = 0.0
            probabilities[:-1] = (1 - probabilities[-1]) / len(potential_error_words)
            k = choice(range(len(potential_error_words) + 1), p=probabilities)
            potential_data.loc[i, 'error_type'] = error_col.replace('_words_errorified', '')
            errorified = potential_data.loc[i, error_col][k]
            potential_data.loc[i, 'error_word'] = errorified
            original = potential_data.loc[i, error_col.replace('_errorified', '')][k]
            potential_data.loc[i, 'error'] = potential_data.loc[i, 'text'].replace(original, errorified)

        potential_data = pd.concat([potential_data[['text', 'error', 'error_word', 'error_type']], 
                                    potential_data_correct[['text', 'error', 'error_word', 'error_type']]], ignore_index=True)
        return potential_data[['text', 'error', 'error_word', 'error_type']]
    
    
    def errorify_sentences(self, data, potential_cols):
        print('Errorifying sentences...')
        np.random.seed(42)
        # if more than 1 error, they may interfere with each other
        # simple setting: only 1 error
        #potential_cols = ['tsya_words_errorified', 'tysya_words_errorified', 'n_words_errorified', 'nn_words_errorified']
#         cols = ['text'] + potential_cols + ['tsya_words', 'tysya_words', 'n_words', 'nn_words', 'techenie_words', 'techenii_words'
#                                                'ne_words', 'ni_words']
        
        cols = ['text'] + potential_cols + [word + '_words' for word in self.words]
        print(cols)
        potential_data = data[cols].copy()
        potential_data['error'] = np.nan
        potential_data['error_type'] = np.nan
        potential_data['error_word'] = np.nan
        potential_data.reset_index(drop=True, inplace=True)
        for i, row in tqdm(potential_data.iterrows()): # slow method, to be replaced
            cols_maybe_errors = [col for col in potential_cols if (row[col] and not np.all(pd.isnull(row[col])))]
            error_col = choice(cols_maybe_errors)
            # take randomly 1 possible incorrect word or none
            # change-in-future: p of 'no error' depends on the number of possible incorrect words
            potential_error_words = potential_data.loc[i, error_col]
            probabilities = np.empty(len(potential_error_words) + 1)
            probabilities[-1] = 0.5
            probabilities[:-1] = (1 - probabilities[-1]) / len(potential_error_words)
            k = choice(range(len(potential_error_words) + 1), p=probabilities)
            if k == len(potential_error_words):
                potential_data.loc[i, 'error_type'] = 'none'
                potential_data.loc[i, 'error_word'] = np.nan
                potential_data.loc[i, 'error'] = potential_data.loc[i, 'text']
            else:
                potential_data.loc[i, 'error_type'] = error_col.replace('_words_errorified', '')
                errorified = potential_data.loc[i, error_col][k]
                potential_data.loc[i, 'error_word'] = errorified
                original = potential_data.loc[i, error_col.replace('_errorified', '')][k]
                potential_data.loc[i, 'error'] = potential_data.loc[i, 'text'].replace(original, errorified)
        return potential_data[['text', 'error', 'error_word', 'error_type']]
    
    def tokenize(self, sentence):
        return BasicTokenizer(do_lower_case=False).tokenize(sentence)

    def tag_in_conll_format(self, data):
        data['tag'] = 'REPLACE_' + data['error_type']
        exceptions = dict()
        column_names = list(data.columns)
        fast_data = data.values.tolist()
        tokens = []
        labels = []
        for i in tqdm(range(len(fast_data))):
            row = fast_data[i]
            error_col_id = column_names.index('error')
            error_word_col_id = column_names.index('error_word')                
            tag_col_id = column_names.index('tag')
            sent_tokens = self.tokenize(row[error_col_id])
            sent_labels = [self.KEEP] * len(sent_tokens)
            if not pd.isnull(row[error_word_col_id]):
                error_word = str(row[error_word_col_id])
                try:
                    error_word_id = sent_tokens.index(error_word)
                    sent_labels[error_word_id] = row[tag_col_id]
                except Exception as e:
                    print(i, e)
                    exceptions[i] = row
            tokens.append(sent_tokens)
            labels.append(sent_labels)
        self.output_num_sentences = len(tokens)
        return tokens, labels, exceptions
    
    def write_dataset_to_file(self, tokens, labels):
        with open(self.to_file, 'w') as f:
            zipped = list(zip(tokens, labels))
            for i in range(len(zipped)):
                sentence_tokens = zipped[i][0]
                sentence_labels = zipped[i][1]
                for j in range(len(sentence_tokens)):
                    f.write(sentence_tokens[j] + '\t' + sentence_labels[j] + '\n')
                f.write('\n')
    
    def generate(self, to_file=None, prepared_path = None):
        if to_file is not None:
            self.to_file = to_file
#         self.data = self.select_data()
#         print('Data size:', len(self.data))
# #         with open('data/all_data_longer_than_10.txt') as f:
# #             self.data = f.read().splitlines()
#         print('Data size:', len(self.data))
#         self.data = self.sentence_tokenize(self.data)
#         print('Data size:', len(self.data))
#         self.data = self.filter_characters(self.data)
#         print('Data size:', len(self.data))
# #         if 'nn' in self.generate_type:
# #             self.data = self.insert_correct_schitanye(self.data)
# #             print('Data size:', len(self.data))
#         self.data.to_csv(prepared_path)
        
        self.data = pd.read_csv(prepared_path, index_col=0)
        print(self.data.info(memory_usage="deep"))
        if 'nn' in self.generate_type:
            self.data = self.add_schitannye(self.data)
            print('Data size:', len(self.data))
        
        if 'tsya' in self.generate_type:
            self.data = self.mark_containing_tsya_tysya(self.data)
            print('Data size:', len(self.data))
        if 'nn' in self.generate_type:
            self.data = self.mark_containing_n_nn(self.data)
            print('Data size:', len(self.data))
        if 'techenie' in self.generate_type:
            self.data = self.mark_containing_techenie_ii(self.data)
            print('Data size:', len(self.data))
        if 'neni_b' in self.generate_type:
            self.data = self.mark_containing_ne_ni(self.data)
            print('Data size:', len(self.data))
        for error_name in self.words:
        #for error_name in ['tsya', 'tysya', 'n', 'nn', 'techenie', 'techenii', 'ne', 'ni']:
            self.data = self.generate_one_error_type(self.data, error_name)
            print('Data size:', len(self.data))
        self.data, potential_cols = self.filter_data_by_error_set(self.data)
        print('Data size:', len(self.data))
        self.data = self.errorify_sentences_new(self.data, potential_cols)
        print('Data size:', len(self.data))
        tokens, labels, exceptions = self.tag_in_conll_format(self.data)
        print('Data size:', len(tokens))
        self.data = self.data.drop(exceptions.keys())
        self.write_dataset_to_file(tokens, labels)
        return self.data, tokens, labels

In [12]:
import re
search = re.compile(r'(\bmany\b)|(\bmuch\b)|(\blot of\b)', re.IGNORECASE)

In [16]:
search.search('Lot of.')

<re.Match object; span=(0, 6), match='Lot of'>

In [42]:
word = 'Title'
if word.istitle():
    word_correct = 'A ' + word[0].lower() + word[1:]
word_correct

'A title'

### Prepare dataset for model retraining

In [38]:
%%time
generator = SyntheticDatasetGenerator(all_data='dataset_for_retrain_3333.txt', output_file = 'data/dataset_for_retrain_3333_with_mistakes_prob_0.5_full.txt', 
                                      generate_type=['tsya', 'nn'], data_dir='')

Loading dictionaries...
CPU times: user 99 ms, sys: 7.99 ms, total: 107 ms
Wall time: 105 ms


In [39]:
%%time
generated_data, tokens, labels = generator.generate(to_file='datasets/dataset_for_retrain_3333_with_mistakes_prob_0.5_full.txt', 
                                                    prepared_path='datasets/prepared_data_for_retrain_3333_with_mistakes_prob_0.5_full.csv')

Selecting lines larger than 10 symbols...
Data size: 17901
Data size: 17901
Splitting in sentences...


5633it [00:00, 56327.10it/s]

Input sentences before splitting on ... : 18036
Input sentences: 18024
Data size: 18024
Filtering characters...


18024it [00:00, 57693.88it/s]


Data size: 18024
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18024 entries, 0 to 18023
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18024 non-null  object
dtypes: object(1)
memory usage: 6.6 MB
None
Marking tsya/tysya sentences...
2571 825
Data size: 18024
Marking n/nn sentences...


...

15456 5472
Data size: 18024
Generating error tsya ...
Data size: 18024
Generating error tysya ...
Data size: 18024
Generating error n ...
Data size: 18024
Generating error nn ...
Data size: 18024
Filtering out the sentences that cannot be errorified...
Output sentences: 13233
Data size: 13233
Errorifying sentences...
['text', 'tsya_words_errorified', 'tysya_words_errorified', 'n_words_errorified', 'nn_words_errorified', 'tsya_words', 'tysya_words', 'n_words', 'nn_words']


13233it [00:08, 1576.54it/s]
  5%|▌         | 680/13233 [00:00<00:01, 6796.88it/s]

Data size: 13233
64 'Канне' is not in list
65 'Канне' is not in list
90 'Гена' is not in list
91 'Гена' is not in list
92 'Гена' is not in list
94 'Гена' is not in list
294 'корены' is not in list
295 'корены' is not in list
296 'корены' is not in list
327 'Гена' is not in list
328 'Гена' is not in list
329 'Гена' is not in list
421 'Донне' is not in list
422 'Донне' is not in list
564 'Гена' is not in list
565 'Гена' is not in list
566 'Гена' is not in list
646 'Донне' is not in list
647 'Донне' is not in list
778 'Гена' is not in list
779 'Гена' is not in list


100%|██████████| 13233/13233 [00:02<00:00, 6442.40it/s]


Data size: 13233
CPU times: user 12.4 s, sys: 25 ms, total: 12.4 s
Wall time: 12.3 s


In [104]:
# 5252
print(len(generated_data))
generated_data.error_type.value_counts()

12472


n        8806
none     2975
nn        322
tsya      232
tysya     137
Name: error_type, dtype: int64

In [101]:
# 3333
print(len(generated_data))
generated_data.error_type.value_counts()

12280


n        8664
none     3000
nn        246
tsya      220
tysya     150
Name: error_type, dtype: int64

In [24]:
# 3333 prob = 0.5
print(len(generated_data))
generated_data.error_type.value_counts()

12281


none     6156
n        5714
nn        165
tsya      150
tysya      96
Name: error_type, dtype: int64

In [31]:
# 2222
print(len(generated_data))
generated_data.error_type.value_counts()

8186


n        5788
none     1976
tsya      163
nn        160
tysya      99
Name: error_type, dtype: int64

### Fix and analyse files with answeres of model on ruwiki

In [3]:
df = pd.read_csv("ruwiki_2018_09_25_answered_fl_2_doubled_sent.csv")
df

,Unnamed: 0,article_uuid,proc_sentence,corrected,error_types,probability
0,1317,27ff3ee6-e469-4f6d-8d89-777ab91aad6b,"Иван, влюбленный в Варвару, решает отправится на поиски царевны вместе со своим новым другом Чудо-Юдом.","Иван, влюбленный в Варвару, решает отправиться на поиски царевны вместе со своим новым другом Чудо-Юдом.",['тся -> ться'],[0.99915826]
1,1485,afd02154-d8bc-4d80-adc8-57986115b203,"А в 1943 году Рита Истомина и Михаил Юрьев устроились на санитарный поезд №95, где днём работали в вагонах за медсестёр и санитарок, работали и на кухни, а вечером шли по вагонам с концертом для раненных, пели песни, танцевали.","А в 1943 году Рита Истомина и Михаил Юрьев устроились на санитарный поезд №95, где днём работали в вагонах за медсестёр и санитарок, работали и на кухни, а вечером шли по вагонам с концертом для раненых, пели песни, танцевали.",['нн -> н'],[0.9847805]
2,1498,afd02154-d8bc-4d80-adc8-57986115b203,"8 марта 1882 года был установлен на средства Нижнеисетского завода памятник императору Александру II- 2-х саженная серая мраморная колонна, на верху которой бронзовый бюст императора, а под ним надпись «Царю-освободителю императору Александру II-му», на обратной стороне «8-го марта 1882 года», в венке «8 марта 1861 г.».","8 марта 1882 года был установлен на средства Нижнеисетского завода памятник императору Александру II- 2-х саженая серая мраморная колонна, на верху которой бронзовый бюст императора, а под ним надпись «Царю-освободителю императору Александру II-му», на обратной стороне «8-го марта 1882 года», в венке «8 марта 1861 г.».",['нн -> н'],[0.5428505]
3,2185,bce48e1a-1d2a-4b8f-9529-cdcf9bc8272c,"Страдая бредом величия и именую себя мессией социал-демократии, в ночь на первое апреля 1876 Майнлендер повесился в своей квартире в Оффенбах-на-Майне, используя стопку копий «Философии освобождения» как подставку.","Страдая бредом величия и именную себя мессией социал-демократии, в ночь на первое апреля 1876 Майнлендер повесился в своей квартире в Оффенбах-на-Майне, используя стопку копий «Философии освобождения» как подставку.","['н -> нн', 'н -> нн']","[0.9860064, 0.89134]"
4,3119,4fa6ec1d-d865-4948-b83f-9958ea4e01f4,"После поездки наставником юного художника становиться Анатолий Дёмин, который направил юношу на учёбу в Ярославль.","После поездки наставником юного художника становится Анатолий Дёмин, который направил юношу на учёбу в Ярославль.",['ться -> тся'],[0.9997673]
...,...,...,...,...,...,...
743,637309,41e9532f-2243-4a92-9e6a-a1a2e05636bf,"По словам лидера группы, это помогает ей сосредоточится и положительно влияет на процесс сочинения.","По словам лидера группы, это помогает ей сосредоточиться и положительно влияет на процесс сочинения.",['тся -> ться'],[0.999123]
744,638200,5211ec13-fd3a-4599-87d9-4b60a95e17ba,"Высокомерно относится к людям, не верит в возможность мирного сосуществования рас, терпеть не может Юки и Зэро, является объектом особого восхищения старосты Дневного Класса, где учится Юки (причем неоднократно его отвергает).","Высокомерно относиться к людям, не верит в возможность мирного сосуществования рас, терпеть не может Юки и Зэро, является объектом особого восхищения старосты Дневного Класса, где учится Юки (причем неоднократно его отвергает).",['тся -> ться'],[0.750137]
745,638271,5211ec13-fd3a-4599-87d9-4b60a95e17ba,"Зэро и Итиру были очень близки в детстве, но после того, как Итиру понял, что ему никогда не сравниться с братом, и случайно услышал, что родители предпочитают сделать наследником клана Зэро, а не его, возненавидел брата.","Зэро и Итиру были очень близки в детстве, но после того, как Итиру понял, что ему никогда не сравнится с братом, и случайно услышал, что родители предпочитают сделать наследником клана Зэро, а не его, возненавидел брата.",['ться -> тся'],[0.9819324]
746,638327,5211ec13-fd3a-4599-87d9-4b60a95e17ba,"Пол ребёнка читателям долгое время был не известен, так как в японской речи он/она говорит о себе обезличено, а на рекламном тизере одной из экстр, а так

In [4]:
dlin = re.compile(r'\bдлин[а|у|ы|е|ой]', flags=re.IGNORECASE)
dlinn = re.compile(r'\bдлинн[а|у|ы|е|ой]', flags=re.IGNORECASE)
#indices_dlinn = df[df['corrected'].str.contains(dlinn)].index.to_numpy()
df[df['corrected'].str.contains(dlinn)]
#df = df[~df.index.isin(indices_dlinn)]

,Unnamed: 0,Unnamed: 0.1,article_uuid,proc_sentence,corrected,error_types,probability


In [4]:
print(df.columns)
df = df.drop(['Unnamed: 0.1'], axis=1)

Index(['Unnamed: 0', 'article_uuid', 'proc_sentence', 'corrected',
       'error_types', 'probability'],
      dtype='object')


KeyError: "['Unnamed: 0.1'] not found in axis"

In [5]:
cols = df.columns.tolist()
cols = ['initial_index'] + cols[1:]
df.columns = cols
df = df.set_index('initial_index')
df.head()

,article_uuid,proc_sentence,corrected,error_types,probability
initial_index,,,,,
1317,27ff3ee6-e469-4f6d-8d89-777ab91aad6b,"Иван, влюбленный в Варвару, решает отправится на поиски царевны вместе со своим новым другом Чудо-Юдом.","Иван, влюбленный в Варвару, решает отправиться на поиски царевны вместе со своим новым другом Чудо-Юдом.",['тся -> ться'],[0.99915826]
1485,afd02154-d8bc-4d80-adc8-57986115b203,"А в 1943 году Рита Истомина и Михаил Юрьев устроились на санитарный поезд №95, где днём работали в вагонах за медсестёр и санитарок, работали и на кухни, а вечером шли по вагонам с концертом для раненных, пели песни, танцевали.","А в 1943 году Рита Истомина и Михаил Юрьев устроились на санитарный поезд №95, где днём работали в вагонах за медсестёр и санитарок, работали и на кухни, а вечером шли по вагонам с концертом для раненых, пели песни, танцевали.",['нн -> н'],[0.9847805]
1498,afd02154-d8bc-4d80-adc8-57986115b203,"8 марта 1882 года был установлен на средства Нижнеисетского завода памятник императору Александру II- 2-х саженная серая мраморная колонна, на верху которой бронзовый бюст императора, а под ним надпись «Царю-освободителю императору Александру II-му», на обратной стороне «8-го марта 1882 года», в венке «8 марта 1861 г.».","8 марта 1882 года был установлен на средства Нижнеисетского завода памятник императору Александру II- 2-х саженая серая мраморная колонна, на верху которой бронзовый бюст императора, а под ним надпись «Царю-освободителю императору Александру II-му», на обратной стороне «8-го марта 1882 года», в венке «8 марта 1861 г.».",['нн -> н'],[0.5428505]
2185,bce48e1a-1d2a-4b8f-9529-cdcf9bc8272c,"Страдая бредом величия и именую себя мессией социал-демократии, в ночь на первое апреля 1876 Майнлендер повесился в своей квартире в Оффенбах-на-Майне, используя стопку копий «Философии освобождения» как подставку.","Страдая бредом величия и именную себя мессией социал-демократии, в ночь на первое апреля 1876 Майнлендер повесился в своей квартире в Оффенбах-на-Майне, используя стопку копий «Философии освобождения» как подставку.","['н -> нн', 'н -> нн']","[0.9860064, 0.89134]"
3119,4fa6ec1d-d865-4948-b83f-9958ea4e01f4,"После поездки наставником юного художника становиться Анатолий Дёмин, который направил юношу на учёбу в Ярославль.","После поездки наставником юного художника становится Анатолий Дёмин, который направил юношу на учёбу в Ярославль.",['ться -> тся'],[0.9997673]


#### Check if some sentences were not fixed by the model

In [6]:
len(df[df.proc_sentence == df.corrected])

7

In [7]:
df = df[df.proc_sentence != df.corrected]
len(df)

741

In [8]:
np.unique(df.error_types.values)

array(["['н -> нн', 'н -> нн']", "['н -> нн']", "['нн -> н']",
       "['тся -> ться', 'тся -> ться']", "['тся -> ться']",
       "['ться -> тся']"], dtype=object)

#### Save fixed dataframe

In [9]:
df.to_csv('ruwiki_2018_09_25_answered_fl_2_doubled_sent.csv')

### Analyse comparisons of models

In [2]:
import pandas as pd

In [3]:
pd.read_csv('comparison_universal_default_incorrect.csv')

,Model,Mihail_acc,Mihail_p,Mihail_r,Mihail_f1,Gramota_acc,Gramota_p,Gramota_r,Gramota_f1,Full_acc,Full_p,Full_r,Full_f1,Tsya_acc,Tsya_p,Tsya_r,Tsya_f1
0,FL_hardsoft_1,0.831081,0.822785,0.855263,0.838710,0.717949,0.701149,0.772152,0.734940,0.774515,0.759036,0.812903,0.785047,0.938053,0.940000,0.921569,0.930693
1,FL_hardsoft_2,0.831081,0.807229,0.881579,0.842767,0.679487,0.655914,0.772152,0.709302,0.755284,0.727273,0.825806,0.773414,0.938053,0.940000,0.921569,0.930693
2,FL_hardsoft_2_70_30,0.837838,0.817073,0.881579,0.848101,0.679487,0.659341,0.759494,0.705882,0.758663,0.734104,0.819355,0.774390,0.951327,0.959596,0.931373,0.945274
3,POW_hardsoft_1,0.831081,0.822785,0.855263,0.838710,0.698718,0.681818,0.759494,0.718563,0.764900,0.748503,0.806452,0.776398,0.924779,0.904762,0.931373,0.917874
4,POW_hardsoft_2,0.837838,0.833333,0.855263,0.844156,0.666667,0.655172,0.721519,0.686747,0.752252,0.739394,0.787097,0.762500,0.920354,0.888889,0.941176,0.914286
5,DP,0.831081,0.822785,0.855263,0.838710,0.673077,0.652174,0.759494,0.701754,0.752079,0.730994,0.806452,0.766871,0.946903,0.932692,0.950980,0.941748


In [3]:
pd.read_csv('comparison_universal_default_incorrect.csv')

,Model,Mihail_acc,Mihail_p,Mihail_r,Mihail_f1,Gramota_acc,Gramota_p,Gramota_r,Gramota_f1,Full_acc,Full_p,Full_r,Full_f1,Tsya_acc,Tsya_p,Tsya_r,Tsya_f1
0,fl_hardsoft_correct_1set,0.850746,0.850746,1.0,0.919355,0.733333,0.733333,1.0,0.846154,0.792040,0.795276,1.0,0.885965,0.876106,0.813559,0.941176,0.872727
1,fl_hardsoft_correct_2set,0.821918,0.821918,1.0,0.902256,0.692308,0.692308,1.0,0.818182,0.757113,0.760870,1.0,0.864198,0.876106,0.813559,0.941176,0.872727
2,fl_hardsoft_correct_2set_70_30,0.833333,0.833333,1.0,0.909091,0.646154,0.646154,1.0,0.785047,0.739744,0.744526,1.0,0.853556,0.884956,0.822034,0.950980,0.881818
3,fl_hardsoft_correct_2set_prob_0.5,0.850746,0.850746,1.0,0.919355,0.734694,0.734694,1.0,0.847059,0.792720,0.801724,1.0,0.889952,0.876106,0.803279,0.960784,0.875000
4,fl_hardsoft_correct__doubled_sent_2set,0.865672,0.865672,1.0,0.928000,0.730769,0.730769,1.0,0.844444,0.798220,0.806723,1.0,0.893023,0.871681,0.796748,0.960784,0.871111
5,fl_hardsoft_correct_2set_retrained_2222_with_m...,0.867647,0.867647,1.0,0.929134,0.689655,0.689655,1.0,0.816327,0.778651,0.785714,1.0,0.880000,0.876106,0.818966,0.931373,0.871560
6,fl_hardsoft_correct_2set_retrained_3333_with_m...,0.892308,0.892308,1.0,0.943089,0.701754,0.701754,1.0,0.824742,0.797031,0.803279,1.0,0.890909,0.876106,0.808333,0.950980,0.873874
7,fl_hardsoft_correct_2set_retrained_5252_with_m...,0.893939,0.893939,1.0,0.944000,0.666667,0.666667,1.0,0.800000,0.780303,0.788618,1.0,0.881818,0.858407,0.791667,0.931373,0.855856
8,fl_hardsoft_correct_2set_retrained_3333_with_m...,0.906250,0.906250,1.0,0.950820,0.706897,0.706897,1.0,0.828283,0.806573,0.811475,1.0,0.895928,0.871681,0.796748,0.960784,0.871111
9,fl_hardsoft_correct_2set_retrained_3332_with_m...,0.867647,0.867647,1.0,0.929134,0.754098,0.754098,1.0,0.859813,0.810873,0.813953,1.0,0.897436,0.871681,0.792000,0.970588,0.872247


### Compare answers of  fl_hardsoft_1_universal & fl_hardsoft_1_old

In [19]:
df = pd.read_csv('compare_fl_hardsoft_1.csv', index_col=0)
#df = df.drop(['Unnamed: 0'], axis=1)
#df.to_csv('compare_fl_hardsoft_1.csv')
#df[~df.fl_hs_1_universal.str.contains('Correct')]
df

,original_sent,fl_hs_1_universal,fl_hs_1_old
0,"Свежий номер ""Комсомольской правды"" к вашему завтраку (количество ежедневных экземпляров ограничено).",Correct,Correct
1,"Свежий номер ""Комсомольской правды"" к вашему завтраку (количество ежедневных экземпляров ограниченно).","Свежий номер ""Комсомольской правды"" к вашему завтраку (количество ежедневных экземпляров ограничено).","Свежий номер ""Комсомольской правды"" к вашему завтраку (количество ежедневных экземпляров ограничено)."
2,Количество призов ограничено - 50 000 шт.,Correct,Correct
3,Количество призов ограниченно - 50 000 шт.,Количество призов ограничено - 50 000 шт.,Количество призов ограничено - 50 000 шт.
4,"Если нет, то почему же внимание автора статей изначально не привлек вопрос, насколько обосновано недовольство и жалобы тех ""не согласных ни с чем граждан"", которые и дали почву для подобных обвинений в мой адрес?","Если нет, то почему же внимание автора статей изначально не привлек вопрос, насколько обоснованно недовольство и жалобы тех ""не согласных ни с чем граждан"", которые и дали почву для подобных обвинений в мой адрес?","Если нет, то почему же внимание автора статей изначально не привлек вопрос, насколько обоснованно недовольство и жалобы тех ""не согласных ни с чем граждан"", которые и дали почву для подобных обвинений в мой адрес?"
...,...,...,...
301,"Что касается конечностей и их длины, в частности: возможно дело не в ""площади потерь тепла"" с конечностей, а в том, что в теплых регионах обмен веществ происходит быстрее, соответственно это отражается на активности,",Correct,Correct
302,Остались считанные дни!,Остались считаные дни!,Остались считаные дни!
303,Остались считаные дни!,Correct,Correct
304,В смысле она вообще не выглядела простужено.,Correct,Correct


### Save all считаные/считанные sentences into files

In [11]:
# data = pd.read_csv('prepared_data_full_endings.csv', index_col=0)
# data2 = pd.read_csv('news_prepared_data_full_endings.csv', index_col=0)
# schitan = re.compile(r'\bсчитан[ые|ыми|ое]', flags=re.IGNORECASE)
# schitann = re.compile(r'\bсчитанн[ые|ыми|ое]', flags=re.IGNORECASE)
# data['n_case'] = data['text'].str.contains(schitan, regex=True)
# data['schitanye'] = data[data['n_case'].fillna(False)]['text']
# data['nn_case']  = data['text'].str.contains(schitann, regex=True)
# data['schitannye'] = data[data['nn_case'].fillna(False)]['text']
# sizes = data.count().to_numpy()
# k = sizes[4]-sizes[2]
# print(k)
# schitan_data = data['schitanye'].dropna().to_numpy()
# schitann_data = data['schitannye'].dropna().to_numpy()
# data2['n_case'] = data2['text'].str.contains(schitan, regex=True)
# data2['schitanye'] = data2[data2['n_case'].fillna(False)]['text']
# data2['nn_case']  = data2['text'].str.contains(schitann, regex=True)
# data2['schitannye'] = data2[data2['nn_case'].fillna(False)]['text']
# sizes = data2.count().to_numpy()
# k2 = sizes[4]-sizes[2]
# print(k2)
# schitan_data2 = data2['schitanye'].dropna().to_numpy()
# schitann_data2 = data2['schitannye'].dropna().to_numpy()
# array1 = np.concatenate((schitan_data, schitann_data), axis=None)
# array2 = np.concatenate((schitan_data2, schitann_data2), axis=None)
# with open("all_shcitanye_sentences_magazines.txt", "w") as f:
#     for sentence in array1:
#         f.write(sentence)
#         f.write('\n')
# with open("all_shcitanye_sentences_news.txt", "w") as f:
#     for sentence in array2:
#         f.write(sentence)
#         f.write('\n')

820


### Save all "Длинна/Длина" sentences into files

In [2]:
# data = pd.read_csv('datasets/prepared_data_hardsoft_correct.csv', index_col=0)
# data2 = pd.read_csv('datasets/news_prepared_data_hardsoft_correct.csv', index_col=0)
# dlin = re.compile(r'\bдлин(а|у|ы|е|(ой))(\b|\.|\,)', flags=re.IGNORECASE)
# dlinn = re.compile(r'\bдлинн(а|у|ы|е|(ой))(\b|\.|\,)', flags=re.IGNORECASE)
# data['n_case'] = data['text'].str.contains(dlin, regex=True)
# data['dlina'] = data[data['n_case'].fillna(False)]['text']
# data['nn_case']  = data['text'].str.contains(dlinn, regex=True)
# data['dlinna'] = data[data['nn_case'].fillna(False)]['text']
# sizes = data.count().to_numpy()
# k = sizes[4]-sizes[2]
# print(k)
# dlin_data = data['dlina'].dropna().to_numpy()
# dlinn_data = data['dlinna'].dropna().to_numpy()
# data2['n_case'] = data2['text'].str.contains(dlin, regex=True)
# data2['dlina'] = data2[data2['n_case'].fillna(False)]['text']
# data2['nn_case']  = data2['text'].str.contains(dlinn, regex=True)
# data2['dlinna'] = data2[data2['nn_case'].fillna(False)]['text']
# sizes = data2.count().to_numpy()
# k2 = sizes[4]-sizes[2]
# print(k2)
# dlin_data2 = data2['dlina'].dropna().to_numpy()
# dlinn_data2 = data2['dlinna'].dropna().to_numpy()
# # To different files for Magazines and News
# # array1 = np.concatenate((dlin_data, dlinn_data), axis=None)
# # array2 = np.concatenate((dlin_data2, dlinn_data2), axis=None)
# # with open("dlina_sentences_magazines.txt", "w") as f:
# #     for sentence in array1:
# #         f.write(sentence)
# #         f.write('\n')
# # with open("dlina_sentences_news.txt", "w") as f:
# #     for sentence in array2:
# #         f.write(sentence)
# #         f.write('\n')

# # To 1 file only "длина" for both sets
# array1 = np.concatenate((dlin_data, dlin_data2), axis=None)
# with open("all_dliNa_sent.txt", "w") as f:
#     for sentence in array1:
#         f.write(sentence)
#         f.write('\n')

/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


65
-1971


In [45]:
print(data.count())
print(sizes)

text       16135507
n_case     16135507
dlina          3436
nn_case    16135507
dlinna         3501
dtype: int64

### Get statistics of generated texts

In [8]:
def get_statistics(input_file, outputfile_tsya, outputfile_nn):
#     print(generated_data.error_word.str.lower().value_counts())
#     df = generated_data.error_word.str.lower().value_counts()
#     df = df.reset_index()
#     cols = ['incorrect_word', 'value_count']
#     df.columns = cols
#     df.to_csv("Statistics_magazines.csv", index=False)
    dict_tsya = {}
    dict_nn = {}
    tsya_data = pd.DataFrame(columns=['word', 'value_count', 'pair_word', 'value_cout_pair'])
    nn_data = pd.DataFrame(columns=['word', 'value_count', 'pair_word', 'value_cout_pair'])

    nn_search = re.compile(r'\wнн([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|'
                                   r'ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b',
                                   re.IGNORECASE)  # the words, which contain "н" in the middle or in the end of word
    n_search = re.compile(r'[аоэеиыуёюя]н([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|'
                                  r'ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b',
                                  re.IGNORECASE)
    pattern_nn = re.compile(
                r'(?-i:нн)'
                r'(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)',
                re.IGNORECASE)
    pattern_n = re.compile(
                r'(?<=[аоэеиыуёюя])(?-i:н)'
                r'(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)',
                re.IGNORECASE)
    
    with open(input_file, 'r') as f:
        line = f.readline().strip()
        line_words = line.split()

        while line:
            if len(line_words) > 0:
                word = line_words[0].lower()
                if re.search('(тся)|(ться)',word) is not None:
                    if word not in dict_tsya:
                        dict_tsya[word] = 1
                    else:
                        dict_tsya[word] += 1
                if n_search.search(word) is not None or nn_search.search(word) is not None:
                    if word not in dict_nn:
                        dict_nn[word] = 1
                    else:
                        dict_nn[word] += 1

            line = f.readline()
            stripped_line = line.strip()
            line_words = stripped_line.split()
    k = 0
    for i,word in enumerate(dict_tsya):
        if 'тся' in word:
            if word.replace('тся', 'ться') in dict_tsya:
                tsya_data.loc[k] = [word, dict_tsya[word], word.replace('тся', 'ться'), dict_tsya[word.replace('тся', 'ться')]]
                k+=1
            else:
                tsya_data.loc[k] = [word, dict_tsya[word], word.replace('тся', 'ться'), 0]
                k+=1
        elif 'ться' in word and word.replace('ться', 'тся') not in dict_tsya:
            tsya_data.loc[k] = [word.replace('ться', 'тся'), 0, word, dict_tsya[word]]
            k+=1
    k = 0
    for i,word in enumerate(dict_nn):
        if n_search.search(word) is not None:
            word_pair = pattern_n.sub('нн', word)
            if word_pair in dict_nn:
                nn_data.loc[k] = [word, dict_nn[word], word_pair, dict_nn[word_pair]]
                k+=1
            else:
                nn_data.loc[k] = [word, dict_nn[word], word_pair, 0]
                k+=1
        elif nn_search.search(word) is not None and pattern_nn.sub('н', word) not in dict_nn:
            nn_data.loc[k] = [pattern_nn.sub('н', word), 0, word, dict_nn[word]]
            k+=1
    nn_data.to_csv(outputfile_nn, index=False)
    tsya_data.to_csv(outputfile_tsya, index=False)

    return tsya_data, nn_data

In [ ]:
tsya_data, nn_data = get_statistics("datasets/dataset_hardsoft_correct.txt", 
                                    "statistics_tsya_magazines.csv", 
                                    "statistics_nn_magazines.csv")

In [ ]:
tsya_data_news, nn_data_news = get_statistics("datasets/news_dataset_hardsoft_correct.txt", 
                                    "statistics_tsya_news.csv", 
                                    "statistics_nn_news.csv")

### Get statistics of pleminary texts

In [88]:
dict_tsya = {}
dict_nn = {}
tsya_data = pd.DataFrame(columns=['word', 'value_count', 'pair_word', 'value_cout_pair'])
nn_data = pd.DataFrame(columns=['word', 'value_count', 'pair_word', 'value_cout_pair'])

nn_search = re.compile(r'\wнн([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|'
                               r'ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b',
                               re.IGNORECASE)  # the words, which contain "н" in the middle or in the end of word
n_search = re.compile(r'[аоэеиыуёюя]н([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|'
                              r'ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b',
                              re.IGNORECASE)
pattern_nn = re.compile(
            r'(?-i:нн)'
            r'(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)',
            re.IGNORECASE)
pattern_n = re.compile(
            r'(?<=[аоэеиыуёюя])(?-i:н)'
            r'(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)',
            re.IGNORECASE)
data = pd.read_csv("datasets/prepared_data_hardsoft_correct.csv", index_col=0)
# tokenizer = BasicTokenizer(do_lower_case=False)
data = data[~data.text.isnull()]
for sentence in data.text.to_numpy():
    word_list = tokenizer.tokenize(sentence)
    for word in word_list:
        word = word.lower()
        if re.search('(тся)|(ться)',word) is not None:
            if word not in dict_tsya:
                dict_tsya[word] = 1
            else:
                dict_tsya[word] += 1
        if n_search.search(word) is not None or nn_search.search(word) is not None:
            if word not in dict_nn:
                dict_nn[word] = 1
            else:
                dict_nn[word] += 1

/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [89]:
k = 0
for i,word in tqdm(enumerate(dict_tsya)):
    if 'тся' in word:
        if word.replace('тся', 'ться') in dict_tsya:
            tsya_data.loc[k] = [word, dict_tsya[word], word.replace('тся', 'ться'), dict_tsya[word.replace('тся', 'ться')]]
            k+=1
        else:
            tsya_data.loc[k] = [word, dict_tsya[word], word.replace('тся', 'ться'), 0]
            k+=1
    elif 'ться' in word and word.replace('ться', 'тся') not in dict_tsya:
        tsya_data.loc[k] = [word.replace('ться', 'тся'), 0, word, dict_tsya[word]]
        k+=1
k = 0
for i,word in tqdm(enumerate(dict_nn)):
    if n_search.search(word) is not None:
        word_pair = pattern_n.sub('нн', word)
        if word_pair in dict_nn:
            nn_data.loc[k] = [word, dict_nn[word], word_pair, dict_nn[word_pair]]
            k+=1
        else:
            nn_data.loc[k] = [word, dict_nn[word], word_pair, 0]
            k+=1
    elif nn_search.search(word) is not None and pattern_nn.sub('н', word) not in dict_nn:
        nn_data.loc[k] = [pattern_nn.sub('н', word), 0, word, dict_nn[word]]
        k+=1
nn_data.to_csv("statistics_nn_magazines_pleminary_data.csv", index=False)
tsya_data.to_csv("statistics_tsya_magazines_pleminary_data.csv", index=False)

### Magazines. tsya + nn, corrected schitanniy sentences

In [13]:
%%time
generator = SyntheticDatasetGenerator(all_data='all_data.txt', output_file = 'data/dataset.txt', generate_type=['tsya', 'nn'])

Loading dictionaries...
CPU times: user 77.8 ms, sys: 32.8 ms, total: 111 ms
Wall time: 109 ms


In [14]:
%%time
generated_data, tokens, labels = generator.generate(to_file='datasets/dataset_hardsoft_correct_doubled_sentences.txt', prepared_path='datasets/prepared_data_hardsoft_correct.csv')

/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16135507 entries, 0 to 16135506
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
dtypes: object(1)
memory usage: 3.7 GB
None
Add schitannye
-1369
Data size: 16136916
Marking tsya/tysya sentences...
1307057 593942
Data size: 16136916
Marking n/nn sentences...


/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


4599270 2599493
Data size: 16136916
Generating error tsya ...
Data size: 16136916
Generating error tysya ...
Data size: 16136916
Generating error n ...
Data size: 16136916
Generating error nn ...
Data size: 16136916
Filtering out the sentences that cannot be errorified...
Output sentences: 1097533
Data size: 1097533
Errorifying sentences...
['text', 'tsya_words_errorified', 'tysya_words_errorified', 'n_words_errorified', 'nn_words_errorified', 'tsya_words', 'tysya_words', 'n_words', 'nn_words']


0it [00:00, ?it/s]

...

1077281 'избавится' is not in list


 50%|█████     | 1097716/2195066 [02:48<02:54, 6287.06it/s]

1096566 'возвратится' is not in list


100%|██████████| 2195066/2195066 [05:36<00:00, 6525.87it/s]


Data size: 2195066
CPU times: user 25min 13s, sys: 18.8 s, total: 25min 32s
Wall time: 25min 19s


In [9]:
tsya_data, nn_data = get_statistics("datasets/dataset_hardsoft_correct2.txt", 
                                    "statistics_tsya_magazines2.csv", 
                                    "statistics_nn_magazines2.csv")

In [17]:
generated_data[generated_data['text'] == "Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно."]

,text,error,error_word,error_type,tag
1,"Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно.","Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмыслено.",осмыслено,nn,REPLACE_nn
1097534,"Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно.","Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно.",NaN,none,REPLACE_none


In [5]:
print(generated_data['error_type'].value_counts())

none     548151
n        221721
tysya    122326
tsya     118893
nn        85033
Name: error_type, dtype: int64


### Magazines dataset. Only tsya/tysya

In [3]:
%%time
generator = SyntheticDatasetGenerator(all_data='all_data.txt', output_file = 'data/dataset.txt', generate_type=['tsya'])

Loading dictionaries...
CPU times: user 6.54 ms, sys: 0 ns, total: 6.54 ms
Wall time: 5.71 ms


In [4]:
%%time
generated_data, tokens, labels = generator.generate(to_file='datasets/dataset_only_tsya.txt', prepared_path='datasets/prepared_data_only_tsya.csv')
# Input sentences should be 15664256 before sentence_tokenize function: correct
# Input sentences should be 16135506: correct
# Output sentences should be 1057067: correct

/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16135506 entries, 0 to 16135505
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
dtypes: object(1)
memory usage: 3.7 GB
None
Marking tsya/tysya sentences...
1306636 593751
Data size: 16135506
Generating error tsya ...
Data size: 16135506
Generating error tysya ...
Data size: 16135506
Filtering out the sentences that cannot be errorified...


0it [00:00, ?it/s]

Output sentences: 498256
Data size: 498256
Errorifying sentences...
['text', 'tsya_words_errorified', 'tysya_words_errorified', 'tsya_words', 'tysya_words']


498256it [05:37, 1475.39it/s]


Data size: 498256


100%|██████████| 498256/498256 [01:26<00:00, 5772.78it/s]


Data size: 498256
CPU times: user 9min 40s, sys: 7.95 s, total: 9min 48s
Wall time: 9min 39s


In [8]:
generated_data.shape

(498256, 5)

### News dataset. tsya + nn, corrected schitanniy sentences

In [18]:
%%time
generator = SyntheticDatasetGenerator(all_data='all_news_dataset.txt', output_file = 'data/news_dataset.txt', generate_type=['tsya', 'nn'])

Loading dictionaries...
CPU times: user 114 ms, sys: 4.04 ms, total: 118 ms
Wall time: 117 ms


In [19]:
%%time
generated_data_news, tokens, labels = generator.generate(to_file='datasets/news_dataset_hardsoft_correct_doubled_sentences.txt', prepared_path='datasets/news_prepared_data_hardsoft_correct.csv')

/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990976 entries, 0 to 3990975
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
dtypes: object(1)
memory usage: 1.2 GB
None
Add schitannye
-384
Data size: 3991362
Marking tsya/tysya sentences...
514831 167124
Data size: 3991362
Marking n/nn sentences...


/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


1742209 911734
Data size: 3991362
Generating error tsya ...
Data size: 3991362
Generating error tysya ...
Data size: 3991362
Generating error n ...
Data size: 3991362
Generating error nn ...
Data size: 3991362
Filtering out the sentences that cannot be errorified...
Output sentences: 473034
Data size: 473034
Errorifying sentences...
['text', 'tsya_words_errorified', 'tysya_words_errorified', 'n_words_errorified', 'nn_words_errorified', 'tsya_words', 'tysya_words', 'n_words', 'nn_words']


0it [00:00, ?it/s]

...

427519 'вооружиться' is not in list


 50%|████▉     | 472806/946068 [01:09<01:06, 7120.25it/s]

472038 'вооружатся' is not in list


100%|██████████| 946068/946068 [02:22<00:00, 6657.89it/s]


Data size: 946068
CPU times: user 10min 12s, sys: 5.98 s, total: 10min 17s
Wall time: 10min 15s


In [45]:
# old
generated_data_news.shape

(472628, 5)

In [13]:
generated_data_news.shape

(472636, 5)

In [12]:
print(generated_data_news['error_type'].value_counts())

none     236567
n        138534
tsya      53379
tysya     32163
nn        11993
Name: error_type, dtype: int64


In [14]:
tsya_data_news, nn_data_news = get_statistics("datasets/news_dataset_hardsoft_correct2.txt", 
                                    "statistics_tsya_news2.csv", 
                                    "statistics_nn_news2.csv")

In [16]:
nn_data_news

,word,value_count,pair_word,value_cout_pair
0,организовано,522,организованно,546
1,возбуждено,9178,возбужденно,8602
2,ына,49,ынна,0
3,районе,4097,районне,0
4,направлена,644,направленна,599
...,...,...,...,...
27819,нетленой,0,нетленной,1
27820,ромалдана,1,ромалданна,0
27821,монотоная,0,монотонная,1
27822,тилькина,1,тилькинна,0


### Download News dataset and save to one file

In [34]:
# archive_url = 'http://bit.ly/2pvhWZm'
# filename = wget.download(archive_url, os.path.join('./data', 'news' + '.zip'))
# pyunpack.Archive('./data/news.zip').extractall('./data')

# file_list_full = []
# for folder in ['Fontanka', 'Lenta', 'KP', 'Interfax']:
#     file_list_year = []
#     for year_folder in os.listdir('./news/' + folder + '/texts/'):
#         file_list = sorted(glob.glob(os.path.join('./news/'+folder + '/texts/' + year_folder + '/', '*.txt')))
#         file_list_year.append(file_list)
#     file_list_full.append(file_list_year)
    
# with open('./data/all_news_dataset.txt', 'w') as file:
#     for file_list_year in file_list_full:
#         for file_list in file_list_year:
#             input_lines = fileinput.input(file_list)
#             file.writelines(input_lines)

In [32]:
# corpus = pd.read_csv('./data/Fontanka/metatable_2017.csv', sep='\t', encoding='utf8')
# os.listdir('news/')

### Generated data analysis

In [24]:
(generated_data['error_type'] == 'none').sum() / len(generated_data)

0.24959948471290627

In [25]:
(generated_data_news['error_type'] == 'none').sum() / len(generated_data_news)

0.25006982235500225

In [26]:
generated_data

,text,error,error_word,error_type,tag
0,"В недавней вашей прозе о Слуцком* вы так описываете его поэтику: 'хриплое клокотание'; 'страшная и обыденная жизнь и смерть, страшно обыденная и обыденно страшная'; 'разговорная речь с вкрапленьями профессионального и бытового жаргона'; 'небрежный (как бы) или иронический тон высказывания и тут же рядом — речь ораторская, поддержанная высокой архаикой вплоть до церковнославянизмов'; 'речевой сплав горнего и дольнего' — и далее, как бы итожа: 'преткновенная гармония', 'лиро-эпос'...","В недавней вашей прозе о Слуцком* вы так описываете его поэтику: 'хриплое клокотание'; 'страшная и обыденная жизнь и смерть, страшно обыденная и обыденно страшная'; 'разговорная речь с вкрапленьями профессионального и бытового жаргонна'; 'небрежный (как бы) или иронический тон высказывания и тут же рядом — речь ораторская, поддержанная высокой архаикой вплоть до церковнославянизмов'; 'речевой сплав горнего и дольнего' — и далее, как бы итожа: 'преткновенная гармония', 'лиро-эпос'...",жаргонна,n,REPLACE_n
1,"Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно.","Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно.",NaN,none,REPLACE_none
2,"О. Ч. К любым языковым формулам надо относиться осторожно, хотя и сказано энергично.","О. Ч. К любым языковым формулам надо относится осторожно, хотя и сказано энергично.",относится,tysya,REPLACE_tysya
3,"Для кого-то так относиться к замыслу важно — тому же Бродскому, который писал достаточно протяженные, синтаксически развернутые вещи.","Для кого-то так относится к замыслу важно — тому же Бродскому, который писал достаточно протяженные, синтаксически развернутые вещи.",относится,tysya,REPLACE_tysya
4,"Картина хороша, но это жанр.","Картинна хороша, но это жанр.",Картинна,n,REPLACE_n
...,...,...,...,...,...
1096085,"Душно в комнате было, и на стене картина, как форточка, открывалась,","Душно в комнате было, и на стене картинна, как форточка, открывалась,",картинна,n,REPLACE_n
1096086,"Посмотри – все дробится в его узоре, и на самой границе небес и вод обнажается детский затылок моря, закругляется горизонт.","Посмотри – все дробится в его узоре, и на самой границе небес и вод обнажается детский затылок моря, закругляется горизонт.",NaN,none,REPLACE_none
1096087,"Иногда асфальт становится на дыбы, корни и грибы растут из него.","Иногда асфальт становиться на дыбы, корни и грибы растут из него.",становиться,tsya,REPLACE_tsya
1096088,Вымерзают глубины до дна.,Вымерзают глубинны до дна.,глубинны,n,REPLACE_n


### Double the test data

In [12]:
# for test_file in ["test_nn/Gramota", "test_nn/Michail_collection"]:
#     with open(test_file + '.txt', 'r') as file:
#         with open(test_file + '_doubled' + '.txt', 'w') as f:
#             text = [line.strip() for line in file]
#             for sentence in text:
#                 sentence, _,sign = sentence.rpartition(',')
#                 f.write(sentence + ',' + sign + '\n')
#                 if sign == "+":
#                     f.write(sentence + ',' + '-' + '\n')
#                 else:
#                     f.write(sentence + ',' + '+' + '\n')
# test_file = "test_nn/Mozhno_itak_itak"
# with open(test_file + '.txt', 'r') as file:
#     with open(test_file + '_doubled' + '.txt', 'w') as f:
#         text = [line.strip() for line in file]
#         for sentence in text:
#             f.write(sentence + '\n')
#             f.write(sentence + '\n')

## Todo

### Ошибки, которые мы взяли в работу

1. Заменить a на b:
    - -ться <-> -тся: просто ищем тся **[РЕАЛИЗОВАНО]**
    - н <-> нн (совершенно, длина): н/нн + список окончаний (см. ниже) **[РЕАЛИЗОВАНО]**
    - течение <-> течении: просто ищем
    - ни- <-> не-: не-ни в начале слова
2. Вставить/убрать пробел: имеет ли смысл в рандомных местах? Наверное, не очень пока
    - ни * <-> ни* (никто)
    - то же <-> тоже
    - что бы <-> чтобы (и чтоб)
    - бестолку <-> бес толку
    - (сложные слова) мега масштабы <-> мегамасштабы
3. Вставить/убрать дефис:
    - из-за <-> из за
    - *-то <-> * то
    - *-таки <-> * таки
4. Комбинация (1) и (2):
    - не кто <-> никто

## Ошибки, которые мы пока не взяли в работу:
1. Опечатки: вставить/заменить букву
    - ве(ч)ером, в(а)ши, бес колебаний, белее того, без везти, ...
2. Вставить предлог
    - письмо (с) фотографиями, ...

### Tagging scheme

#### Replace operations

- replace_tsya
- replace_tysya
- replace_n
- replace_nn
- replace_techenii
- replace_techenie
- replace_ni
- replace_ne

#### Merge and Split operations

- merge: ни кто -> никто
- split: не весел -> невесел
- merge_hyphen: из за -> из-за
- split_hyphen: я-таки -> я таки
- merge_replace_ni: не кто -> никто
- merge_replace_ne:
- split_replace_ni:
- split_replace_ne: 

### Списки окончаний

1. **Список окончаний прилагательных и причастий с н/нн**:  
-ый, -ого, -ому, -ом, -ым, -ая, -ой, -ую, -ые, -ыми, -ых, -а, -о, -ы  
На всякий случай мягкий вариант (лингвисты сказали не делать, но, может, всё-таки добавить на всякий случай?):  
ий|его|ему|ем|им|яя|ей|юю|ие|ими|их|я|е|и  
и пустое окончание, но нам оно не нужно, потому что там гласный вставляется: "пустынен"